#                                    --> PIPELINE PROJECT <--

In [ ]:
# importar modulos necesarios

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

1. Acquisition

In [ ]:
# cargar el dataset elegido

avocados = pd.read_csv('avocado.csv')


In [ ]:
avocados.shape

2. Wrangling

In [ ]:
# comprobar los datos nulos del data set

avocados.isnull().sum()  # No hay nulos en ninguna columna

In [ ]:
avocados.columns

In [ ]:
# Comprobar que no hay duplicados
before = len(avocados)

select_columns = ['Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region']

avocados = avocados[select_columns].drop_duplicates()

after = len(avocados)

print('Numero de registros duplicados: ', str(before - after))

In [ ]:
# Extraer el mes de la columna fecha y desechar el resto
month = []
for e in avocados["Date"]:
    aux= e.split("-")
    month.append(aux[1])
    
avocados['month'] = pd.Series(month, index=avocados.index) # Añadir columna Month al dataframe
avocados['Total Volume']= (avocados['Total Volume']/1000)

In [ ]:
# Cambio del orden de las columnas

column_order = ['region','year','month','type','AveragePrice','Total Volume','4046','4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

avocados = avocados[column_order]

In [ ]:
# comprobar el tipo de datos de las columnas. cambiar year a objeto

avocados['year'] = avocados['year'].astype('object') 

In [ ]:
# renombrar algunas columnas

avocados.columns = ['Region','Year','Month','Variety','Avg Price','Total Volume-k',
                    'S-M size','L size','XL size','Total Bags','S bags','L bags','XL bags']

In [ ]:
# Guardo un csv con los datos manipulados

avocados.to_csv('manipulated_avocados.csv',index=False)

3.Analysis

HIPÓTESIS:
- La variedad de aguacate convencional tendrá un precio medio inferior al aguacate orgánico.
- Las regiones donde el precio medio sea más alto presentarán un volument total de venta menor
- El % que aporten al volumen total (aguacates S-M, aguacates L, aguacares XL y bolsas (mezcla tamaños) tendrán relación con el precio medio.


In [ ]:
# Dado que del año 2018 solo hay registros hasta marzo voy a eliminar esos datos
filteredyear = list(avocados[(avocados['Year']==2018)].index) 
avocados = avocados.drop(filteredyear, axis=0)


In [ ]:
# Las regiones vienen reflejadas todas juntas de varias maneras: Total US, grandes regiones y por estados.
# Voy a separar en 3 dataframe: Uno de regiones, otro del total y otro de ciudades

avocados_region = avocados[(avocados['Region'] == 'Plains')|(avocados['Region'] =='West')|
                    (avocados['Region'] =='California')|(avocados['Region'] =='SouthCentral')|
                    (avocados['Region'] =='Northeast')|(avocados['Region'] =='Southeast')|
                    (avocados['Region'] =='GreatLakes')|(avocados['Region'] =='Midsouth')]
avocados_totalUS = avocados[(avocados['Region'] == 'TotalUS')]

avocados_cities= avocados[(avocados['Region'] != 'Plains')|(avocados['Region'] !='West')|
                    (avocados['Region'] !='California')|(avocados['Region'] !='SouthCentral')|
                    (avocados['Region'] !='Northeast')|(avocados['Region'] !='Southeast')|
                    (avocados['Region'] !='GreatLakes')|(avocados['Region'] !='Midsouth')|
                   (avocados['Region'] != 'TotalUS')]

In [ ]:
#VARIEDAD vs AVG PRICE (En todo US)

variedad_precio =avocados_region.groupby(['Variety','Year'])['Avg Price'].describe()
variedad_precio 
# Efectivamente el aguacate orgánico es más caro que el convencional. 
# Por año se observa que se ha incrementado el precio medio en 2017. Aunque no es lineal, en 2016 disminuyó.

In [ ]:
#VARIEDAD vs AVG PRICE (por regiones y año)

variedad_precio_region = pd.pivot_table(avocados_region, values=['Avg Price'], index=['Region'], columns=['Year'])

variedad_precio_region

In [ ]:
# VARIEDAD vs. TOTAL VOLUMEN (todo US por año y mes)

year_month_volume = pd.pivot_table(avocados_totalUS, values=['Total Volume-k'], index=['Month'], columns=['Year'])

year_month_volume

In [ ]:
#VARIEDAD vs TOTALVOLUME (por regiones y año)

variedad_volumen_region = pd.pivot_table(avocados_region, values=['Total Volume-k'], index=['Region'], columns=['Year'])

variedad_volumen_region

In [ ]:
avocados_region['Total_profit-k']= (avocados_region['Avg Price']*avocados_region['Total Volume-k'])
total_profit_region = pd.pivot_table(avocados_region, values=['Total_profit-k'], index=['Region'], columns=['Year']).copy()
total_profit_region

In [ ]:
total_volumen_region = pd.pivot_table(avocados_region, values=['Total_profit-k','Total Volume-k'], index=['Region'], columns=['Year'])
total_volumen_region

In [ ]:
correlacion1 = avocados_region['Total Volume-k'].corr(avocados_region['Avg Price'])
correlacion1

In [ ]:
new_columns = ['perc_S-M size','perc_L size','perc_XL size','perc_Bags','perc_S bags','perc_L bags','perc_XL bags']
origin_columns = ['S-M size','L size','XL size','Total Bags','S bags','L bags','XL bags']
for n, o in zip(new_columns, origin_columns):
    avocados_region[n] = (avocados_region[o]/avocados_region['Total Volume-k'])/10

percen_vol= avocados_region[['Region','Year','perc_S-M size','perc_L size','perc_XL size','perc_Bags']].copy()
percen_vol = pd.pivot_table(percen_vol, values=['perc_S-M size','perc_L size','perc_XL size','perc_Bags'], index=['Region'],columns=['Year'])

print("Porcentajes de aporte al volumen total:\n{}".format(percen_vol))

4. Visualizacion

In [ ]:
import matplotlib.pyplot as plt  
    
title = 'Total volumen de venta por mes (2015, 2016 y 2017)'

fig, ax = plt.subplots(figsize=(15,8))
barchart = sns.barplot(data=avocados_totalUS, x=('Month'), y='Total Volume-k')
plt.title(title + "\n", fontsize=16)

In [ ]:
title = 'Precio medio de venta'

fig, ax = plt.subplots(figsize=(15,8))
barchart = sns.barplot(data=avocados_totalUS, x=('Month'), y='Avg Price')
plt.title(title + "\n", fontsize=16)

In [ ]:
sns.boxplot(y="Variety", x="Avg Price", data=avocados, palette = 'Purples')